In [7]:
# Imports
import subprocess
import pathlib
import os
import logging
import sys
import shutil
from datetime import datetime

In [8]:
from utils import find_compound_name_by_cid

In [9]:
# Logging setup
time_start = datetime.now()
log_filename = f"{time_start.year}-{str(time_start.month).zfill(2)}-{str(time_start.day).zfill(2)}_{str(time_start.hour).zfill(2)}_{str(time_start.minute).zfill(2)}_{str(time_start.second).zfill(2)}"

os.makedirs("./logs/", exist_ok = True)

with open(f"./logs/{log_filename}.log", 'w', encoding = 'utf-8') as f:
    f.truncate(0)

logging.basicConfig(level = 'INFO', format = '[%(asctime)s] [%(thread)d - %(threadName)s] [%(levelname)s] %(message)s', encoding = 'utf-8',
                    handlers = [
                        logging.FileHandler(f"./logs/{log_filename}.log", encoding = 'utf-8'),
                        logging.StreamHandler(stream=sys.stdout)
                    ])

In [10]:
os.makedirs('./ligands', exist_ok=True)
os.makedirs('./receptors', exist_ok=True)

shutil.rmtree('./vina', ignore_errors=True)
os.makedirs('./vina', exist_ok=True)

shutil.rmtree('./output', ignore_errors=True)
os.makedirs('./output', exist_ok=True)

In [11]:
# Configs
vina_path = pathlib.Path('C:/Users/NatsumeRyuhane/Desktop/vina_1.2.5_win.exe')
ligands_path = pathlib.Path('./ligands')
receptors_path = pathlib.Path('./receptors')

class VinaParams:
    def __init__(self, center_x, center_y, center_z, size_x, size_y, size_z, exhaustiveness = 8, num_modes = 9, cpu = 16):
        self.center_x = center_x
        self.center_y = center_y
        self.center_z = center_z
        self.size_x = size_x
        self.size_y = size_y
        self.size_z = size_z
        self.cpu = cpu
        
        # how thorough the search will be. Higher values increase the search thoroughness, 
        # but also increase the time it takes to complete the docking
        # The default value is 8
        self.exhaustiveness = exhaustiveness
        
        # how many binding modes will be generated
        # The default value is 9
        self.num_modes = num_modes

paramSetDict = {
    "MIRO1mono_model1": VinaParams(
        center_x = 0.0,
        center_y = 0.0,
        center_z = 0.0,
        size_x = 20.0,
        size_y = 20.0,
        size_z = 20.0
    )
}

logging.info(f'Initialization Complete')

[2024-10-13 02:04:50,849] [975256 - MainThread] [INFO] Initialization Complete


In [12]:
receptors = list(receptors_path.glob('*.pdbqt'))
ligands = list(ligands_path.glob('*.pdbqt'))
logging.info(f'Found {len(receptors)} Receptors, {len(ligands)} Ligands')

# get all receptor files
for receptor in receptors:
    receptor_name = receptor.stem
    logging.info(f'Receptor: {receptor_name}')
    os.makedirs(f'./output/{receptor_name}', exist_ok=True)

    if receptor_name not in paramSetDict:
        logging.error(f"No defined parameters found for receptor: {receptor_name}")
        continue
    else:
        paramSet = paramSetDict.get(receptor_name)
        
    # get all ligand files
    for ligand in ligands:
        ligand_cid = ligand.stem
        ligand_name = find_compound_name_by_cid(ligand_cid)
        
        if ligand_name is None:
            logging.warning(f"Compound name not found for CID: {ligand_cid}")
            ligand_name = "ligandNameNotFound"
        else:
            logging.info(f'Ligand: {ligand_name}')
            
        receptor_file = receptor
        ligand_file = ligand
        
        with open(f'./output/{receptor_name}/{ligand_name}_{ligand_cid}.pdbqt', 'w') as f:
            pass
        
        # run vina
        command = [
            str(vina_path),
            '--receptor', str(receptor_file),
            '--ligand', str(ligand_file),
            '--center_x', str(paramSet.center_x),
            '--center_y', str(paramSet.center_y),
            '--center_z', str(paramSet.center_z),
            '--size_x', str(paramSet.size_x),
            '--size_y', str(paramSet.size_y),
            '--size_z', str(paramSet.size_z),
            '--exhaustiveness', str(paramSet.exhaustiveness),
            '--num_modes', str(paramSet.num_modes),
            '--cpu', str(paramSet.cpu),
            '--out', f'./output/{receptor_name}/{ligand_name}_{ligand_cid}.pdbqt'  # Output file for docking results
        ]
        try:
            result = subprocess.run(command, cwd="./", check=True, capture_output=True, text=True)
            
            stdout = result.stdout
            
            # TODO: Post-process the output
            with open(f'./output/{receptor_name}/{ligand_name}_{ligand_cid}.log', 'w') as f:
                f.write(stdout)
            
            stderr = result.stderr
            logging.info(f"Completed: {ligand_name} - {receptor_name}")
            
            if stderr:
                logging.warning(f"<STDERR>: {stderr}")
        except subprocess.CalledProcessError as e:
            nl = '\n'
            logging.warning(f"Vina finished with return code {e.returncode}.\n\n<STDERR>{nl + '| ' + str(e.stderr).replace(nl, nl + '| ')}\n</STDERR>\n")
            
            # Assuming the output file is useless if the process failed and should be removed
            os.remove(f'./output/{receptor_name}/{ligand_name}_{ligand_cid}.pdbqt')
            continue
        except Exception as e:
            logging.error(f"Unexpected Error: {e}")
            continue

[2024-10-13 02:04:50,866] [975256 - MainThread] [INFO] Found 1 Receptors, 6 Ligands
[2024-10-13 02:04:50,866] [975256 - MainThread] [INFO] Receptor: MIRO1mono_model1
[2024-10-13 02:04:50,867] [975256 - MainThread] [INFO] Ligand: 3-Hydroxybenzaldehyde
[2024-10-13 02:04:50,916] [975256 - MainThread] [WARNING] Vina finished with return code 1.

<STDERR>
| Command line parse error: too many positional options have been specified on the command line
| 
| Correct usage:
| 
| Input:
|   --receptor arg             rigid part of the receptor (PDBQT)
|   --flex arg                 flexible side chains, if any (PDBQT)
|   --ligand arg               ligand (PDBQT)
|   --batch arg                batch ligand (PDBQT)
|   --scoring arg (=vina)      scoring function (ad4, vina or vinardo)
| 
| Search space (required):
|   --maps arg                 affinity maps for the autodock4.2 (ad4) or vina 
|                              scoring function
|   --center_x arg             X coordinate of the center 